In [ ]:
TWITTER SENTIMENT ANALYSIS 

In [ ]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class TwitterClient(object):
    #generic twitter class for sentiment analysis
    def __init__(self):
        consumer_key='' #just copy paste your consumer key
        consumer_secret=''#here goes your consumer secret
        access_token_key=''#your access token keys
        access_token_secret=''#access token secret keys

        #attempting authentication
        try:
            #create OAuthHandler object
            self.auth = OAuthHandler(consumer_key,consumer_secret)

            #set access token and secret
            self.auth.set_access_token(access_token_key,access_token_secret)

            #create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication failed, Check your keys!!")

    def clean_tweet(self,tweet):
        #applying regex for cleaning the tweets:
        return ' '.join(re.sub(r"(@[a-zA-Z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet).split())
        
    def get_tweet_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet)) #creating textblob object
        #check sentiment
        if analysis.sentiment.polarity > 0:
            return 'pos'
        elif analysis.sentiment.polarity ==0:
            return 'neut'
        else:
            return 'neg'

    def get_tweets(self,query,count):
    #main function to get tweets and parse through them
        tweets=[] #empty list to store tweets

        try:
            #call twitter api to fetch tweets
            fetched_tweets  = self.api.search(q=query,count = count)

            for tweet in fetched_tweets:
                #empty dictionary to store  tweets with corresponding sentiment
                parsed_tweet = {}

                #saving text of a tweet
                parsed_tweet['text'] = tweet.text
                #saving sentiment of a tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                #appending parsed tweets to tweets list
                if tweet.retweet_count>0:
                    #if tweet has retweets , ensure it is appende only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
            return tweets
        except tweepy.TweepError as e: 
            print("Error :    "+str(e))


def main():
    #creating object of TwitterClient class
    obj = TwitterClient()
    #taking a string input for analysis
    q=input("enter your keyword to check for its sentiment analysis: ")

    #calling function to get tweets
    tweets = obj.get_tweets(query = q,count=200)

    #picking positive tweets from the tweets
    ptweets=[tweet for tweet in tweets if tweet['sentiment']=='pos']

    #percentage of positive tweets
    print("% of (+)ve  tweets: {} %".format(100*len(ptweets)//len(tweets)))

    #picking negative tweets from the tweets
    ntweets=[tweet for tweet in tweets if tweet['sentiment']=='neg']

    #percentage of negative tweets
    print("% of (-)ve  tweets: {} %".format(100*len(ntweets)//len(tweets)))

    #percentage of neutral tweets
    print("% of neutral tweets : {}% ".format(100*(len(tweets) - (len(ntweets)+len(ptweets)))//len(tweets)))

    #printing first five positive and negative tweets:
    print("\n\n______POSITIVE TWEETS_____:\n")
    for tweet in ptweets[:100]:
        print(tweet['text'])

    print("\n\n______NEGATIVE TWEETS_____:\n")
    for tweet in ntweets[:100]:
        print(tweet['text'])
     #plotting a bar graph to visualize the analysis   
    sentiment=['POSITIVE','NEGATIVE','NEUTRAL']
    vals=[(100*len(ptweets)/len(tweets)),(100*len(ntweets)/len(tweets)),(100*((len(tweets) - (len(ntweets)+len(ptweets)))/len(tweets)))]
    

    plt.bar(sentiment,vals)
    plt.show() #showing the result on a graph

if __name__=="__main__":
    main()
    
